<a href="https://colab.research.google.com/github/Natalia409634/Grupo7-practica2-ML/blob/main/G7_P2_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PRÁCTICA 2 -- APRENDIZAJE AUTOMÁTICO

# Librerías

In [ ]:
import pandas as pd
import pickle
import os

# Lectura de Datos

Los datos están en formato 'pkl'

In [24]:


# Directorio que contiene los archivos pkl
dir_path = '/content/drive/MyDrive/Aprendizaje Automático/practica 2/datos_grupos/'

#diccionario para almacenar los DataFrames
data_dict = {}

# Lee cada archivo pkl en el directorio y guarda el DataFrame correspondiente en el diccionario
for filename in os.listdir(dir_path):
    if filename.endswith('.pkl'):
        filepath = os.path.join(dir_path, filename)
        key = filename[:-4]  # Elimina la extensión .pkl del nombre de archivo
        data_dict[key] = pd.read_pickle(filepath)


In [39]:
import os
import pandas as pd


# Lista para almacenar los DataFrames
dfs = []

# Lee cada archivo pkl en el directorio y guarda el DataFrame correspondiente en la lista
for filename in os.listdir('/content/drive/MyDrive/Aprendizaje Automático/practica 2/datos_grupos/'):
    if filename.endswith('.pkl'):
        filepath = os.path.join(dir_path, filename)
        df = pd.read_pickle(filepath)
        dfs.append(df)

# Concatena los DataFrames en uno solo
data_df = pd.concat(dfs)

Edición visual de los datos

In [42]:
def set_frame_style(df, caption=""):
    """Helper function to set dataframe presentation style.
    """
    return df.style.background_gradient(cmap='Blues').set_caption(caption).set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'darkblue'),
        ('font-size', '18px'),
        ('font-weight','bold')
    ]}])

# EDA Simplificado

Hacer un EDA muy simplificado: 

*   cuántas instancias / cuantos
atributos y de qué tipo (numéricos, ordinales, categóricos) DONE
*   columnas constantes o
innecesarias DONE
*   que proporción de missing values por atributo
*   tipo de problema:
(clasificación o regresión) -> clasificación no?
*   ¿es desbalanceado?









In [43]:
data_df.head()

,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
1,7.315971,13.0,2.0,4.0,3.0,2.0,4.0,31.0,Yes,NaN,...,0.0,Y,23.0,8.0,NaN,NaN,3.0,5.0,NaN,4.0
6,6.450877,NaN,3.0,4.0,1.0,3.0,1.0,28.0,Yes,Travel_Rarely,...,2.0,Y,20.0,8.0,1.0,5.0,2.0,NaN,0.0,NaN
13,NaN,14.0,2.0,3.0,NaN,2.0,2.0,NaN,Yes,Non-Travel,...,1.0,NaN,11.0,NaN,NaN,10.0,4.0,10.0,9.0,9.0
28,10.713066,6.0,2.0,3.0,NaN,3.0,1.0,NaN,Yes,Travel_Frequently,...,NaN,Y,14.0,8.0,1.0,19.0,2.0,1.0,0.0,0.0
30,9.662808,11.0,NaN,3.0,1.0,2.0,3.0,26.0,Yes,Travel_Rarely,...,NaN,Y,11.0,8.0,0.0,5.0,NaN,3.0,0.0,2.0


In [44]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 441000 entries, 1 to 4409
Data columns (total 31 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   hrs                      372828 non-null  float64
 1   absences                 364050 non-null  float64
 2   JobInvolvement           365856 non-null  float64
 3   PerformanceRating        374143 non-null  float64
 4   EnvironmentSatisfaction  365240 non-null  float64
 5   JobSatisfaction          372361 non-null  float64
 6   WorkLifeBalance          364689 non-null  float64
 7   Age                      363065 non-null  float64
 8   Attrition                441000 non-null  object 
 9   BusinessTravel           368909 non-null  object 
 10  Department               373374 non-null  object 
 11  DistanceFromHome         369199 non-null  float64
 12  Education                368388 non-null  float64
 13  EducationField           367527 non-null  object 
 14  Employ

EL conjunto de datos proporcionado tiene 30 atributos, los cuales adoptan un formato float y un formato 'object'

Identificar y tratar datos faltantes

In [46]:
data_df.isnull().sum()

hrs                        68172
absences                   76950
JobInvolvement             75144
PerformanceRating          66857
EnvironmentSatisfaction    75760
JobSatisfaction            68639
WorkLifeBalance            76311
Age                        77935
Attrition                      0
BusinessTravel             72091
Department                 67626
DistanceFromHome           71801
Education                  72612
EducationField             73473
EmployeeCount              74954
EmployeeID                 70096
Gender                     77256
JobLevel                   73342
JobRole                    72273
MaritalStatus              70357
MonthlyIncome              74078
NumCompaniesWorked         71535
Over18                     73327
PercentSalaryHike          70620
StandardHours              71533
StockOptionLevel           74999
TotalWorkingYears          77778
TrainingTimesLastYear      74296
YearsAtCompany             76616
YearsSinceLastPromotion    70454
YearsWithC

Attition tiene valor 0. Dado que parece un dato irregular, estudiaremos más a fondo esta variable para entender que puede estar pasando

La función dropna elimina filas y columnas que tienen valores nulos. GUardamos el resultado en una nueva variable para poder trabajar con los datos modificados.

In [53]:
data_df.dropna()


,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
120,8.600742,14.0,3.0,4.0,1.0,4.0,2.0,34.0,Yes,Non-Travel,...,8.0,Y,25.0,8.0,1.0,9.0,3.0,5.0,0.0,4.0
2218,6.908168,5.0,3.0,3.0,1.0,1.0,3.0,49.0,Yes,Travel_Frequently,...,3.0,Y,18.0,8.0,0.0,20.0,2.0,4.0,1.0,3.0
2682,8.180052,12.0,3.0,3.0,2.0,4.0,1.0,41.0,Yes,Travel_Rarely,...,8.0,Y,12.0,8.0,1.0,8.0,2.0,6.0,0.0,5.0
160,5.726737,22.0,4.0,4.0,1.0,4.0,1.0,26.0,No,Travel_Rarely,...,1.0,Y,24.0,8.0,0.0,3.0,2.0,3.0,0.0,2.0
286,7.638302,20.0,2.0,4.0,4.0,3.0,2.0,47.0,No,Travel_Frequently,...,9.0,Y,22.0,8.0,1.0,28.0,6.0,22.0,11.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,6.268158,16.0,3.0,3.0,4.0,3.0,2.0,48.0,No,Travel_Rarely,...,4.0,Y,18.0,8.0,0.0,13.0,4.0,0.0,0.0,0.0
1929,6.950258,5.0,3.0,3.0,3.0,2.0,3.0,27.0,No,Travel_Rarely,...,9.0,Y,17.0,8.0,2.0,4.0,2.0,2.0,2.0,2.0
2976,9.546104,12.0,3.0,3.0,2.0,4.0,2.0,37.0,No,Travel_Frequently,...,1.0,Y,14.0,8.0,0.0,4.0,3.0,4.0,1.0,2.0
3251,6.881342,18.0,1.0,3.0,1.0,1.0,3.0,36.0,No,Non-Travel,...,1.0,Y,18.0,8.0,1.0,10.0,2.0,10.0,0.0,9.0


In [59]:
disp_df = data_df.dropna()

In [60]:
disp_df.drop_duplicates()

,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
120,8.600742,14.0,3.0,4.0,1.0,4.0,2.0,34.0,Yes,Non-Travel,...,8.0,Y,25.0,8.0,1.0,9.0,3.0,5.0,0.0,4.0
2218,6.908168,5.0,3.0,3.0,1.0,1.0,3.0,49.0,Yes,Travel_Frequently,...,3.0,Y,18.0,8.0,0.0,20.0,2.0,4.0,1.0,3.0
2682,8.180052,12.0,3.0,3.0,2.0,4.0,1.0,41.0,Yes,Travel_Rarely,...,8.0,Y,12.0,8.0,1.0,8.0,2.0,6.0,0.0,5.0
160,5.726737,22.0,4.0,4.0,1.0,4.0,1.0,26.0,No,Travel_Rarely,...,1.0,Y,24.0,8.0,0.0,3.0,2.0,3.0,0.0,2.0
286,7.638302,20.0,2.0,4.0,4.0,3.0,2.0,47.0,No,Travel_Frequently,...,9.0,Y,22.0,8.0,1.0,28.0,6.0,22.0,11.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,6.350194,21.0,2.0,3.0,3.0,3.0,3.0,42.0,No,Travel_Rarely,...,7.0,Y,16.0,8.0,0.0,10.0,2.0,5.0,0.0,3.0
1929,6.950258,5.0,3.0,3.0,3.0,2.0,3.0,27.0,No,Travel_Rarely,...,9.0,Y,17.0,8.0,2.0,4.0,2.0,2.0,2.0,2.0
2976,9.546104,12.0,3.0,3.0,2.0,4.0,2.0,37.0,No,Travel_Frequently,...,1.0,Y,14.0,8.0,0.0,4.0,3.0,4.0,1.0,2.0
3251,6.881342,18.0,1.0,3.0,1.0,1.0,3.0,36.0,No,Non-Travel,...,1.0,Y,18.0,8.0,1.0,10.0,2.0,10.0,0.0,9.0


DUDA-> HE ELIMINADO LAS VARIABLES REPETIDAS-> NO SE SI HAY QUE AHCERLO YA QUE HAY DEMASIADAS Y ME PARECE RARO

In [61]:
cols = disp_df.columns.to_list()

display(set_frame_style(disp_df[cols].head(),'Primeros'))

display(set_frame_style(disp_df[cols].describe(),'Resumen de estadistica'))

display(set_frame_style(disp_df[cols].isna().any().to_frame().transpose(), 'Columnas con valor nulo'))

display(set_frame_style(disp_df[cols].nunique().to_frame().rename({0:'Numero de valores únicos'}, axis=1).transpose(), 'Valores únicos en cada variable'))

,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
120,8.600742,14.000000,3.000000,4.000000,1.000000,4.000000,2.000000,34.000000,Yes,Non-Travel,Research & Development,7.000000,3.000000,Life Sciences,1.000000,121.000000,Male,1.000000,Sales Representative,Single,56170.000000,8.000000,Y,25.000000,8.000000,1.000000,9.000000,3.000000,5.000000,0.000000,4.000000
2218,6.908168,5.000000,3.000000,3.000000,1.000000,1.000000,3.000000,49.000000,Yes,Travel_Frequently,Sales,6.000000,5.000000,Medical,1.000000,2219.000000,Female,2.000000,Healthcare Representative,Single,23420.000000,3.000000,Y,18.000000,8.000000,0.000000,20.000000,2.000000,4.000000,1.000000,3.000000
2682,8.180052,12.000000,3.000000,3.000000,2.000000,4.000000,1.000000,41.000000,Yes,Travel_Rarely,Research & Development,7.000000,2.000000,Life Sciences,1.000000,2683.000000,Female,1.000000,Research Scientist,Single,20990.000000,8.000000,Y,12.000000,8.000000,1.000000,8.000000,2.000000,6.000000,0.000000,5.000000
160,5.726737,22.000000,4.000000,4.000000,1.000000,4.000000,1.000000,26.000000,No,Travel_Rarely,Research & Development,3.000000,2.000000,Medical,1.000000,161.000000,Female,4.000000,Healthcare Representative,Divorced,24360.000000,1.000000,Y,24.000000,8.000000,0.000000,3.000000,2.000000,3.000000,0.000000,2.000000
286,7.638302,20.000000,2.000000,4.000000,4.000000,3.000000,2.000000,47.000000,No,Travel_Frequently,Research & Development,27.000000,3.000000,Life Sciences,1.000000,287.000000,Male,1.000000,Research Scientist,Divorced,54410.000000,9.000000,Y,22.000000,8.000000,1.000000,28.000000,6.000000,22.000000,11.000000,13.000000


,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,DistanceFromHome,Education,EmployeeCount,EmployeeID,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
count,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000,1825.000000
mean,7.284506,12.965479,2.700822,3.149041,2.729315,2.701918,2.728767,36.718356,8.924932,2.902466,1.000000,2230.817534,2.055890,65771.780822,2.616986,15.216986,8.000000,0.784110,11.250959,2.799452,6.930959,2.166575,4.066301
std,1.324099,5.422441,0.724209,0.356227,1.085764,1.085653,0.725000,9.003266,7.850813,1.024545,0.000000,1278.999599,1.077659,47278.326306,2.500359,3.607533,0.000000,0.851643,7.665894,1.290697,5.909928,3.168111,3.563720
min,5.424757,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,18.000000,1.000000,1.000000,1.000000,2.000000,1.000000,10090.000000,0.000000,11.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.260962,8.000000,2.000000,3.000000,2.000000,2.000000,2.000000,30.000000,2.000000,2.000000,1.000000,1096.000000,1.000000,29420.000000,1.000000,12.000000,8.000000,0.000000,6.000000,2.000000,3.000000,0.000000,1.000000
50%,6.977738,13.000000,3.000000,3.000000,3.000000,3.000000,3.000000,35.000000,7.000000,3.000000,1.000000,2229.000000,2.000000,49990.000000,1.000000,14.000000,8.000000,1.000000,10.000000,3.000000,5.000000,1.000000,3.000000
75%,7.894093,17.000000,3.000000,3.000000,4.000000,4.000000,3.000000,42.000000,13.000000,4.000000,1.000000,3350.000000,3.000000,87890.000000,4.000000,18.000000,8.000000,1.000000,15.000000,3.000000,9.000000,3.000000,7.000000
max,10.881625,24.000000,4.000000,4.000000,4.000000,4.000000,4.000000,60.000000,29.000000,5.000000,1.000000,4405.000000,5.000000,199990.000000,9.000000,25.000000,8.000000,3.000000,40.000000,6.000000,40.000000,15.000000,17.000000


,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
Numero de valores únicos,1490,24,4,2,4,4,4,43,2,3,3,29,5,6,1,1491,2,5,9,3,971,10,1,15,1,4,40,7,37,16,18


¿Problema de clasificación o de regresión?

Se trat de un problema de clasificación, ya que la variable de salida que se espera del modelo es categórica, es decir, se espera que el modelo prediga si un empleado abandonará o no la empresa en un futuro cercano

**¿El problema está balanceado?**

In [66]:
disp_df.value_counts()

hrs        absences  JobInvolvement  PerformanceRating  EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance  Age   Attrition  BusinessTravel     Department              DistanceFromHome  Education  EducationField   EmployeeCount  EmployeeID  Gender  JobLevel  JobRole                 MaritalStatus  MonthlyIncome  NumCompaniesWorked  Over18  PercentSalaryHike  StandardHours  StockOptionLevel  TotalWorkingYears  TrainingTimesLastYear  YearsAtCompany  YearsSinceLastPromotion  YearsWithCurrManager
6.063686   12.0      2.0             3.0                3.0                      3.0              3.0              35.0  No         Travel_Rarely      Human Resources         1.0               3.0        Human Resources  1.0            441.0       Male    3.0       Laboratory Technician   Married        136750.0       1.0                 Y       15.0               8.0            3.0               5.0                5.0                    5.0             1.0                      0.0          

HELP con esto jaja

# Evaluación

Simplemente
dividiremos los datos en un conjunto de train para entrenar y ajustar hiper-Aprendizaje Automático
Práctica 2: Predicción de burnout
parámetros, y un conjunto de test en el que evaluaremos las distintas posibilidades
que se probarán en la práctica. Hay que recordar que En problemas de clasificación
desbalanceados hay que usar particiones estratificadas y métricas adecuadas
(balanced_accuracy, f1, matriz de confusión). También es conveniente que los
métodos de construcción de modelos traten el desbalanceo, usando por ejemplo
el parámetro class_weight=”balanced”.

# Construcción de modelos

Construcción de modelos: para esta práctica usaremos
LogisticRegression como método base (sin ajustar hiper-parámetros) y Boosting
como método avanzado (ajustando hiper-parámetros), a elegir. Es importante
realizar los preprocesos que los datos necesiten, usando preferentemente
pipelines. Como método de boosting, se puede elegir uno de entre los métodos de
boosting disponibles en scikit-learn. Si además se usa uno de entre las librerías
externas xgboost, lightgbm o catboost, se pueden sacar +0.35 puntos adicionales.

# Mejora de resultados

Usando algún método de selección de atributos de tipo filter
(SelectKBest) de entre los disponibles en sklearn (f_classif,
mutual_info_classif o chi2), comprobad si se pueden mejorar los resultados del
apartado anterior y extraer conclusiones sobre qué atributos son más importantes,
al menos de acuerdo a estos métodos.